In [2]:
import pandas as pd
import numpy as np
import datetime
from multiprocessing import Pool
import types


#Read file and select relative columns
fire_data = pd.read_csv('FireData.csv')
climate_data = pd.read_csv('ClimateData.csv')
climate_data.columns = ['Station','Date','Air Temperature(Celcius)','Relative Humidity','WindSpeed (knots)','Max Wind Speed','MAX','MIN','Precipitation']
fire_selected_col = fire_data[['Date','Datetime','Surface Temperature (Celcius)','Confidence']]

In [3]:
#Convert to date type
climate_selected_col = climate_data[['Date','Air Temperature(Celcius)']]
climate_selected_col['Date'] = pd.to_datetime(climate_selected_col.Date)

/home/deep/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Parallel searching algorithm for range selection
def s_hash(x, n):
    ### START CODE HERE ###
    result = x%n
    ### END CODE HERE ###
    return result

def parallel_search_range(data, query_range, n_processor):
    rr = []
    pool = Pool(processes=n_processor)
    ### START CODE HERE ###
    dic = {} # We will use a dictionary
    for i, x in enumerate(data['Confidence']): # For each data record, perform the following
        h = s_hash(x, n_processor) # Get the hash key of the input
        if (h in dic.keys()): # If the key exists
            l = dic[h]
            l.append((i,x))
            dic[h] = l # Add the new input to the value set of the key
        else: # If the key does not exist
            l = [] # Create an empty value set
            l.append((i,x))
            dic[h] = l # Add the value set to the key

    for i in range(query_range[0],query_range[1]):
        s=s_hash(i,n_processor)
        if s in dic.keys():
            for j in dic[s]:
                if j[-1]==i:
                    rr.append(j[0])
                    
    return data.loc[rr]

In [140]:
#filtered by confidence with in 80 - 101
filtered = parallel_search_range(fire_selected_col,[80,101],4)
filtered['Date'] = pd.to_datetime(filtered.Date)
filtered = filtered.sort_values(by=['Date'])
filtered = filtered.reset_index(drop = True)

In [6]:
from datetime import datetime
def range_partition(data, range_indices):
    result = []
    n_bin = len(range_indices) 
    # For each bin, perform the following
    for i in range(n_bin): 
        s = data.loc[data['Date'] < range_indices[i]]
        result.append(s) 
        data = data.loc[data['Date'] >= range_indices[i]]
    result.append(data.loc[data['Date'] >= range_indices[n_bin-1]]) 
    return result

In [7]:
#Test range_partition
date_divisions = ['2017/4/1','2017/7/1','2017/10/1']
date_divisions = pd.to_datetime(date_divisions)
S = range_partition(filtered,date_divisions)
R = range_partition(climate_selected_col,date_divisions)

In [91]:
def H(r):
    """
    We define a hash function 'H' that is used in the hashing process works 
    by summing the first and second digits of the hashed attribute, which
    in this case is the join attribute. 
    
    Arguments:
    r -- a record where hashing will be applied on its join attribute

    Return:
    result -- the hash index of the record r
    """
    # Convert the value of the join attribute into the digits
    digits = [int(d) for d in str(r[0]) if d.isdigit()]
    
    # Calulate the sum of elemenets in the digits
    return sum(digits)
def HB_join(T1, T2):
    
    """
    Perform the hash-based join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined

    Return:
    result -- the joined table
    """
    
    result = []
    
    dic = {} # We will use a dictionary
    
    # For each record in table T2
    for s in T2.index:
        # Hash the record based on join attribute value using hash function H into hash table
        s_key = H(T2.loc[s])
        if s_key in dic:
            dic[s_key].append(T2.loc[s]) # If there is an entry
        else:
            dic[s_key] = [T2.loc[s]] 
        
    # For each record in table T1 (probing)
    for r in T1.index:
        # Hash the record based on join attribute value using H
        r_key = H(T1.loc[r])
        

        # If an index entry is found Then
        if r_key in dic:
            
            # Compare each record on this index entry with the record of table T1
            for item in dic[r_key]:

                if item[0] == T1.loc[r][0]:

                    # Put the rsult
                    result.append([T1.loc[r][0],T1.loc[r][1],item[0],item[1],item[2],item[3]])
    
    return result

In [136]:
# Include this package for parallel processing
import multiprocessing as mp

def DPBP_join(T1, T2, n_processor):
    """
    Perform a disjoint partitioning-based parallel join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined
    n_processor -- the number of parallel processors

    Return:
    result -- the joined table
    """
    
    result = []
    
    ### START CODE HERE ### 
    
    # Partition T1 & T2 into sub-tables using rr_partition().
    # The number of the sub-tables must be the equal to the n_processor
    T1_subsets = range_partition(T1, date_divisions)
    T2_subsets = range_partition(T2, date_divisions)
    
    # Pool: a Python method enabling parallel processing. 
    pool = mp.Pool(processes = n_processor)
    
    for i in range(len(T1_subsets)):
        # Apply a join on each processor
        result.extend(pool.apply(HB_join, [T1_subsets[i], T2_subsets[i]]))

    ### END CODE HERE ###
    
    return result

In [137]:
n_processor = 4
result = DPBP_join(climate_selected_col, filtered, n_processor)

In [139]:
output = pd.DataFrame(result,columns = list(climate_selected_col.columns) +list(filtered.columns))